In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
!pip install -q torchinfo
from torchinfo import summary

In [4]:
## fully connected layer will loss spatial information, NiN use (1x1 kernel) as dropout and fully connected layer

class NiN(nn.Module):
    def __init__(self):
        super().__init__()


    def nin_block(self, in_channels, out_channels, kernel_size, strides, padding):
        return nn.Sequential(
                      nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
                      nn.ReLU(),
                      nn.Conv2d(out_channels, out_channels, kernel_size=1), 
                      nn.ReLU(),
                      nn.Conv2d(out_channels, out_channels, kernel_size=1), 
                      nn.ReLU()
              ) 

    def forward(self, x):
        result = nn.Sequential(
            self.nin_block(1, 96, kernel_size=11, strides=4, padding=0),
            nn.MaxPool2d(3, stride=2),
            self.nin_block(96, 256, kernel_size=5, strides=1, padding=2),
            nn.MaxPool2d(3, stride=2),
            self.nin_block(256, 384, kernel_size=3, strides=1, padding=1),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout(0.5),
            self.nin_block(384, 10, kernel_size=3, strides=1, padding=1),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten())

        

        return result(x)


net = NiN() 

In [5]:
summary(model=net, 
        input_size=(1, 1, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
NiN (NiN)                                [1, 1, 224, 224]     [1, 10]              --                   --
Total params: 0
Trainable params: 0
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.20
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.20